**Эксперимент с дополнительной предобработкой тренировочных данных**

In [44]:
import json
import os
import sys

import joblib

# Добавляем корневую директорию проекта в sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

# Импортируем metrics из utils
from utils import metrics as mtc
from utils import preprocess_functions as ppf


In [45]:
import numpy as np
import pandas as pd
import random


# Готовим дата-сеты (предобработка)

In [46]:
reference = pd.read_csv("../../data/preprocessed/general/reference.csv")
train = pd.read_csv("../../data/splited/train.csv")
valid = pd.read_csv("../../data/splited/valid.csv")

abbr_dict = joblib.load("../../resources/general/abbreviations_dict.joblib")
region_dict = joblib.load("../../resources/general/region_dict.joblib")
blacklist_opf = joblib.load("../../resources/general/blacklist_opf.joblib")


In [47]:
reference["processed_name"] = reference.name.apply(
    ppf.simple_preprocess_text
).str.lower()
reference.processed_name = reference.processed_name.apply(ppf.replace_numbers_with_text)
reference.processed_name = reference.processed_name.apply(
    ppf.abbr_preprocess_text, args=(abbr_dict, False, False, True, False)
)
reference.processed_name = reference.processed_name.apply(
    ppf.process_region, args=(list(region_dict.keys()),)
)
reference.processed_name = reference.processed_name.apply(
    ppf.remove_substrings, args=(blacklist_opf,)
)
reference.head(1)


,id,name,region,processed_name
0,69,Кировска,мурманская область,кировска


In [48]:
valid["processed_name"] = valid.name.apply(ppf.simple_preprocess_text).str.lower()
valid.processed_name = valid.processed_name.apply(ppf.replace_numbers_with_text)
valid.processed_name = valid.processed_name.apply(
    ppf.abbr_preprocess_text, args=(abbr_dict, False, False, True, False)
)
valid["region"] = valid.processed_name.apply(
    ppf.process_region, args=(list(region_dict.keys()), True)
)

valid.processed_name = valid.processed_name.apply(
    ppf.process_region, args=(list(region_dict.keys()),)
)
valid.processed_name = valid.processed_name.apply(
    ppf.remove_substrings, args=(blacklist_opf,)
)
valid.head(1)


,name,school_id,processed_name,region
0,КФК Е. Герасимовой,1843,клуб фигурного катания герасимовой,None


In [49]:
train["processed_name"] = train.name.apply(ppf.simple_preprocess_text).str.lower()
train.processed_name = train.processed_name.apply(ppf.replace_numbers_with_text)
train.processed_name = train.processed_name.apply(
    ppf.abbr_preprocess_text, args=(abbr_dict, False, False, True, False)
)
train["region"] = train.processed_name.apply(
    ppf.process_region, args=(list(region_dict.keys()), True)
)

train.processed_name = train.processed_name.apply(
    ppf.process_region, args=(list(region_dict.keys()),)
)
train.processed_name = train.processed_name.apply(
    ppf.remove_substrings, args=(blacklist_opf,)
)
train.head(1)


,name,school_id,processed_name,region
0,"ХМАО-Югра, СШ г. Покачи",173,спортивная школа покачи,ханты мансийский автономный округ


# Данные для обучения

In [50]:
merged_train = train.join(
    reference.set_index("id"), on="school_id", how="left", rsuffix="_reference"
)
merged_train = merged_train[~merged_train.name_reference.isna()]
merged_train = merged_train[~merged_train.processed_name.isna()]
# merged_train = merged_train[
#     [
#         "processed_name",
#         "region",
#         "processed_name_reference",
#         "region_reference",
#     ]
# ]
merged_train["target"] = 1


In [51]:
merged_train.head()

,name,school_id,processed_name,region,name_reference,region_reference,processed_name_reference,target
0,"ХМАО-Югра, СШ г. Покачи",173,спортивная школа покачи,ханты мансийский автономный округ,СШ,ханты мансийский автономный округ,спортивная школа,1
1,"Санкт-Петербург, СПб ГБУ СШОР по фигурному кат...",198,спортивная школа олимпийского резерва по фигур...,санкт петербург,СПБ ГБУ ДО СШОР «Академия ФКК»,санкт петербург,спортивная школа олимпийского резерва академия...,1
2,"ЯНАО, ГАУ ДО ЯНАО СШ ""Ямал""",267,спортивная школа ямал,ямало ненецкий автономный округ,Ямал,ямало ненецкий автономный округ,ямал,1
3,"МО Смольнинское, СФК Ice Dream ИП Карлинской",5,смольнинское секция фигурного катания ice drea...,None,Ice Dream / Айс Дрим,санкт петербург,ice dream айс дрим,1
4,"Москва, МАФК школа ""Олимп""",119,московская академия фигурного катания на коньк...,москва,Олимп,москва,олимп,1


In [52]:
for i, row in merged_train.iterrows():
    print(f"Запись {i}:")
    print(f"Left - {row['name']}")
    print(f"Left - {row['processed_name']}")
    print(f"Left- {row['region']}")
    print(f"Right - {row['processed_name_reference']}")
    print(f"Right - {row['name_reference']}")
    print(f"Right- {row['region_reference']}")
    print()


Запись 0:
Left - ХМАО-Югра, СШ г. Покачи
Left - спортивная школа покачи
Left- ханты мансийский автономный округ
Right - спортивная школа
Right - СШ
Right- ханты мансийский автономный округ

Запись 1:
Left - Санкт-Петербург, СПб ГБУ СШОР по фигурному катанию на конька
Left - спортивная школа олимпийского резерва по фигурному катанию на конька
Left- санкт петербург
Right - спортивная школа олимпийского резерва академия фигурного катания на коньках
Right - СПБ ГБУ ДО СШОР «Академия ФКК»
Right- санкт петербург

Запись 2:
Left - ЯНАО, ГАУ ДО ЯНАО СШ "Ямал"
Left - спортивная школа ямал
Left- ямало ненецкий автономный округ
Right - ямал
Right - Ямал
Right- ямало ненецкий автономный округ

Запись 3:
Left - МО Смольнинское, СФК Ice Dream ИП Карлинской
Left - смольнинское секция фигурного катания ice dream  карлинской
Left- None
Right - ice dream айс дрим
Right - Ice Dream / Айс Дрим
Right- санкт петербург

Запись 4:
Left - Москва, МАФК школа "Олимп"
Left - московская академия фигурного катания 